In [6]:
import sys
import os
import pandas as pd
import scanpy as sc
from scipy import sparse
import numpy as np
import matplotlib.pyplot as plt
import squidpy as sq
import spatialdm as sdm
import seaborn as sns
import pickle

In [2]:
from copulacci import cci
from copulacci import spatial
from copulacci import model
from copulacci import simulation
from copulacci import model2

In [9]:
parent_data = '/n/fs/ragr-data/datasets/spatial_transcriptomics/human_liver_visium'

In [3]:
def linearize_to_df(cop_df_diff_gpairs, filter_copula=True):
    df = pd.DataFrame()
    for gpair in cop_df_diff_gpairs:
        tmp = cop_df_diff_gpairs[gpair].copy()
        if tmp.shape[0] == 0:
            continue
        if filter_copula:
            tmp = tmp.loc[tmp.copula_method == 'copula',:].copy()
        if tmp.shape[0] == 0:
            continue 
        tmp.loc[:,'interaction'] = gpair
        tmp.loc[:,'LR_index'] = tmp.index
        tmp.index = list(range(tmp.shape[0]))
        df = pd.concat([df, tmp], axis = 0, ignore_index=True)
    return df

In [17]:
def prepare_liver_visium(sample_name):
    print(sample_name + " -- processing..." )
    adata = sc.read_h5ad(os.path.join(parent_data, sample_name))
    adata.obs['celltype'] = adata.obs.cell_type.values
    # Change gene names to symbols
    map_file = '/n/fs/ragr-research/users/hirak/Projects/copulacci/notebook/examples/grch38_ensgene_symbol.csv'
    mapping_dict = pd.read_csv(map_file, index_col=0).reset_index().set_index('gene_ids')['index'].to_dict()
    adata.var['gene_symbols'] = adata.var.index.map(mapping_dict)
    adata.var['ensembl_ids'] = adata.var.index
    adata.var.index = adata.var['gene_symbols']
    return adata

In [20]:
sample_dict = {
    'cholangitis' : 'sclerosing_cholangitis_patient_PSC011_C1.h5ad',
    'healthy' : 'healthy_liver_C73_A1.h5ad'
}
adata_dict = {}

print("====Preprocessing===")

for sample_name in sample_dict:
    # For this sample
    adata_dict[sample_name] = prepare_liver_visium(sample_dict[sample_name])

====Preprocessing===
sclerosing_cholangitis_patient_PSC011_C1.h5ad -- processing...
healthy_liver_C73_A1.h5ad -- processing...


,mapped_reference_assembly,mapped_reference_annotation,alignment_software,donor_id,self_reported_ethnicity_ontology_term_id,donor_living_at_sample_collection,organism_ontology_term_id,sample_uuid,sample_preservation_method,tissue_ontology_term_id,...,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,observation_joinid,celltype
AAACAAGTATCTCCCA-1,GRCh38,GENCODE 32,Space Ranger v1.1.0,PSC011,unknown,True,NCBITaxon:9606,ce83db34-64ae-4600-9637-630e3b0cb0e7,OCT embedding,UBERON:0001117,...,centrilobular region hepatocyte,Visium Spatial Gene Expression,primary sclerosing cholangitis,Homo sapiens,female,caudate lobe of liver,unknown,human adult stage,gFD!+Zqd(i,centrilobular region hepatocyte
AAACAGAGCGACTCCT-1,GRCh38,GENCODE 32,Space Ranger v1.1.0,PSC011,unknown,True,NCBITaxon:9606,ce83db34-64ae-4600-9637-630e3b0cb0e7,OCT embedding,UBERON:0001117,...,native cell,Visium Spatial Gene Expression,primary sclerosing cholangitis,Homo sapiens,female,caudate lobe of liver,unknown,human adult stage,-$<USDZJGm,native cell
AAACAGCTTTCAGAAG-1,GRCh38,GENCODE 32,Space Ranger v1.1.0,PSC011,unknown,True,NCBITaxon:9606,ce83db34-64ae-4600-9637-630e3b0cb0e7,OCT embedding,UBERON:0001117,...,native cell,Visium Spatial Gene Expression,primary sclerosing cholangitis,Homo sapiens,female,caudate lobe of liver,unknown,human adult stage,U@Q8P(aq*%,native cell
AAACAGGGTCTATATT-1,GRCh38,GENCODE 32,Space Ranger v1.1.0,PSC011,unknown,True,NCBITaxon:9606,ce83db34-64ae-4600-9637-630e3b0cb0e7,OCT embedding,UBERON:0001117,...,hepatocyte,Visium Spatial Gene Expression,primary sclerosing cholangitis,Homo sapiens,female,caudate lobe of liver,unknown,human adult stage,O3sm{X3xkp,hepatocyte
AAACAGTGTTCCTGGG-1,GRCh38,GENCODE 32,Space Ranger v1.1.0,PSC011,unknown,True,NCBITaxon:9606,ce83db34-64ae-4600-9637-630e3b0cb0e7,OCT embedding,UBERON:0001117,...,centrilobular region hepatocyte,Visium Spatial Gene Expression,primary sclerosing cholangitis,Homo sapiens,female,caudate lobe of liver,unknown,human adult stage,sQx3gJp|)r,centrilobular region hepatocyte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTGTGTGTCAAGA-1,GRCh38,GENCODE 32,Space Ranger v1.1.0,PSC011,unknown,True,NCBITaxon:9606,ce83db34-64ae-4600-9637-630e3b0cb0e7,OCT embedding,UBERON:0001117,...,hepatocyte,Visium Spatial Gene Expression,primary sclerosing cholangitis,Homo sapiens,female,caudate lobe of liver,unknown,human adult stage,?>!di5|#wO,hepatocyte
TTGTTTCACATCCAGG-1,GRCh38,GENCODE 32,Space Ranger v1.1.0,PSC011,unknown,True,NCBITaxon:9606,ce83db34-64ae-4600-9637-630e3b0cb0e7,OCT embedding,UBERON:0001117,...,centrilobular region hepatocyte,Visium Spatial Gene Expression,primary sclerosing cholangitis,Homo sapiens,female,caudate lobe of liver,unknown,human adult stage,!-K_Z3vDq*,centrilobular region hepatocyte
TTGTTTCATTAGTCTA-1,GRCh38,GENCODE 32,Space Ranger v1.1.0,PSC011,unknown,True,NCBITaxon:9606,ce83db34-64ae-4600-9637-630e3b0cb0e7,OCT embedding,UBERON:0001117,...,native cell,Visium Spatial Gene Expression,primary sclerosing cholangitis,Homo sapiens,female,caudate lobe of liver,unknown,human adult stage,&q#Fy^DD*S,native cell
TTGTTTCCATACAACT-1,GRCh38,GENCODE 32,Space Ranger v1.1.0,PSC011,unknown,True,NCBITaxon:9606,ce83db34-64ae-4600-9637-630e3b0cb0e7,OCT embedding,UBERON:0001117,...,periportal region hepatocyte,Visium Spatial Gene Expression,primary sclerosing cholangitis,Homo sapiens,female,caudate lobe of liver,unknown,human adult stage,ZuyRYj_t<2,periportal region hepatocyte


In [8]:
!ls /n/fs/ragr-data/datasets/spatial_transcriptomics/human_liver_visium

healthy_liver_C73_A1.h5ad  sclerosing_cholangitis_patient_PSC011_C1.h5ad


In [ ]:
with open(os.path.join(parent_dir, ''))